In [5]:
# Import libraries
import datetime
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option("display.max_rows", 500)
pd.options.display.max_colwidth = 1000
import seaborn as sns
import time
import warnings
warnings.filterwarnings("ignore")

In [3]:
# Import CSV files
trivago_path = "../data/train.csv"
hotels_path = "../data/hotels_items.csv"

# Create DataFrame
trivago_df = pd.read_csv(trivago_path)
hotels_df = pd.read_csv(hotels_path, usecols = ["item_id", "properties", "city", "price"])

In [6]:
# Convert UNIX time stamp (timestime column) to UTC 
date_conv = lambda x: datetime.datetime.utcfromtimestamp(x).strftime("%Y-%m-%d")
time_conv = lambda x: datetime.datetime.utcfromtimestamp(x).strftime("%H:%M:%S")
trivago_df["date"] = trivago_df["timestamp"].map(date_conv)
trivago_df["time"] = trivago_df["timestamp"].map(time_conv)

In [7]:
# Drop UNIX timestamp
trivago_df.drop("timestamp", axis = 1, inplace = True)